In [ ]:
!pip install torch torchvision pandas scikit-learn matplotlib


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the Breast Cancer Wisconsin Dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
data = pd.read_csv(url, header=None, names=columns)

# Encode target variable: Malignant (M) = 1, Benign (B) = 0
data['Diagnosis'] = data['Diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

# Split features and labels
X = data.iloc[:, 2:].values  # Features (30 columns)
y = data['Diagnosis'].values  # Labels (binary classification)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

# Instantiate the model
model = LogisticRegression(input_dim=30)


In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        # Forward pass
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/50, Loss: 0.5255
Epoch 2/50, Loss: 0.4162
Epoch 3/50, Loss: 0.3596
Epoch 4/50, Loss: 0.3246
Epoch 5/50, Loss: 0.3134
Epoch 6/50, Loss: 0.2693
Epoch 7/50, Loss: 0.2609
Epoch 8/50, Loss: 0.2422
Epoch 9/50, Loss: 0.2253
Epoch 10/50, Loss: 0.2200
Epoch 11/50, Loss: 0.2055
Epoch 12/50, Loss: 0.1987
Epoch 13/50, Loss: 0.1999
Epoch 14/50, Loss: 0.1860
Epoch 15/50, Loss: 0.1842
Epoch 16/50, Loss: 0.1897
Epoch 17/50, Loss: 0.1761
Epoch 18/50, Loss: 0.1686
Epoch 19/50, Loss: 0.1615
Epoch 20/50, Loss: 0.1690
Epoch 21/50, Loss: 0.1570
Epoch 22/50, Loss: 0.1612
Epoch 23/50, Loss: 0.1551
Epoch 24/50, Loss: 0.1530
Epoch 25/50, Loss: 0.1565
Epoch 26/50, Loss: 0.1459
Epoch 27/50, Loss: 0.1597
Epoch 28/50, Loss: 0.1424
Epoch 29/50, Loss: 0.1360
Epoch 30/50, Loss: 0.1391
Epoch 31/50, Loss: 0.1354
Epoch 32/50, Loss: 0.1384
Epoch 33/50, Loss: 0.1304
Epoch 34/50, Loss: 0.1293
Epoch 35/50, Loss: 0.1368
Epoch 36/50, Loss: 0.1310
Epoch 37/50, Loss: 0.1354
Epoch 38/50, Loss: 0.1336
Epoch 39/50, Loss: 0.

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        predictions = (predictions >= 0.5).float()  # Convert probabilities to binary labels
        correct += (predictions == y_batch).sum().item()
        total += y_batch.size(0)

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 98.25%


In [6]:
# First cell - Install just the required packages
!pip install torch>=1.7.0
!pip install torchvision>=0.9.1
!pip install omegaconf>=2.0.6
!pip install onnx>=1.7.0
!pip install pandas>=1.2.2
!pip install pyyaml>=5.3.1
!pip install tensorboard
!pip install future
!pip install scipy>=1.6.0
!pip install scikit-learn  # Modern replacement for sklearn

!pip install crypten --no-deps  # Install crypten without extra dependencies


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 4.9 MB/s eta 0:00:00


In [17]:

# Import libraries
import crypten
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Initialize CrypTen
crypten.init()

# Define the MPC workflow using the @crypten.mpc.run_multiprocess decorator
@crypten.mpc.run_multiprocess(world_size=3)
def train_mpc_model():
    # Load the Breast Cancer Wisconsin Dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
    data = pd.read_csv(url, header=None, names=columns)

    # Encode target variable: Malignant (M) = 1, Benign (B) = 0
    data['Diagnosis'] = data['Diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

    # Split features and labels
    X = data.iloc[:, 2:].values  # Features (30 columns)
    y = data['Diagnosis'].values  # Labels (binary classification)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Partition training data among 3 parties
    X_party1, X_temp, y_party1, y_temp = train_test_split(X_train, y_train, test_size=0.67, random_state=42)
    X_party2, X_party3, y_party2, y_party3 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Convert data to PyTorch tensors
    X_party1_tensor = torch.tensor(X_party1, dtype=torch.float32)
    y_party1_tensor = torch.tensor(y_party1, dtype=torch.float32).unsqueeze(1)
    X_party2_tensor = torch.tensor(X_party2, dtype=torch.float32)
    y_party2_tensor = torch.tensor(y_party2, dtype=torch.float32).unsqueeze(1)
    X_party3_tensor = torch.tensor(X_party3, dtype=torch.float32)
    y_party3_tensor = torch.tensor(y_party3, dtype=torch.float32).unsqueeze(1)

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Encrypt data using CrypTen for 3 parties
    encrypted_X_party1 = crypten.cryptensor(X_party1_tensor, src=0)
    encrypted_y_party1 = crypten.cryptensor(y_party1_tensor, src=0)

    encrypted_X_party2 = crypten.cryptensor(X_party2_tensor, src=1)
    encrypted_y_party2 = crypten.cryptensor(y_party2_tensor, src=1)

    encrypted_X_party3 = crypten.cryptensor(X_party3_tensor, src=2)
    encrypted_y_party3 = crypten.cryptensor(y_party3_tensor, src=2)

    # Combine encrypted data
    encrypted_X_train = crypten.cat([encrypted_X_party1, encrypted_X_party2, encrypted_X_party3], dim=0)
    encrypted_y_train = crypten.cat([encrypted_y_party1, encrypted_y_party2, encrypted_y_party3], dim=0)

    # Define logistic regression model using PyTorch
    class LogisticRegression(torch.nn.Module):
        def __init__(self, input_dim):
            super(LogisticRegression, self).__init__()
            self.linear = torch.nn.Linear(input_dim, 1)
            self.sigmoid = torch.nn.Sigmoid()

        def forward(self, x):
            return self.sigmoid(self.linear(x))

    # Create and encrypt the model
    model = LogisticRegression(input_dim=30)
    encrypted_model = crypten.nn.from_pytorch(model, torch.empty(1, 30))
    encrypted_model.encrypt()

    # Define loss function
    loss_fn = crypten.nn.BCELoss()

    # Set precision and learning parameters
    crypten.encoder.fixed_point_precision = 32
    learning_rate = 0.1
    num_epochs = 50

    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        predictions = encrypted_model(encrypted_X_train)
        loss = loss_fn(predictions, encrypted_y_train)

        # Backward pass
        encrypted_model.zero_grad()
        loss.backward()

        # Update parameters with learning rate decay
        lr = learning_rate / (1 + epoch * 0.1)  # Decay learning rate
        encrypted_model.update_parameters(lr)

        # Print decrypted loss for monitoring
        print(f"Epoch {epoch + 1}, Loss: {loss.get_plain_text().item():.4f}")

    # Evaluate the model
    encrypted_model.eval()
    with torch.no_grad():
        predictions = encrypted_model(crypten.cryptensor(X_test_tensor, src=0))
        decrypted_predictions = predictions.get_plain_text()
        binary_predictions = (decrypted_predictions >= 0.5).float()
        accuracy = (binary_predictions == y_test_tensor).float().mean().item()

    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Run the MPC model training
train_mpc_model()

/usr/local/lib/python3.10/dist-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)


Epoch 1, Loss: 0.7247
Epoch 1, Loss: 0.7247Epoch 1, Loss: 0.7247

Epoch 2, Loss: 0.5724Epoch 2, Loss: 0.5724Epoch 2, Loss: 0.5724


Epoch 3, Loss: 0.4949Epoch 3, Loss: 0.4949Epoch 3, Loss: 0.4949


Epoch 4, Loss: 0.4488Epoch 4, Loss: 0.4488Epoch 4, Loss: 0.4488


Epoch 5, Loss: 0.4177Epoch 5, Loss: 0.4177
Epoch 5, Loss: 0.4177

Epoch 6, Loss: 0.3939
Epoch 6, Loss: 0.3939
Epoch 6, Loss: 0.3939
Epoch 7, Loss: 0.3760Epoch 7, Loss: 0.3760

Epoch 7, Loss: 0.3760
Epoch 8, Loss: 0.3617Epoch 8, Loss: 0.3617
Epoch 8, Loss: 0.3617

Epoch 9, Loss: 0.3496Epoch 9, Loss: 0.3496Epoch 9, Loss: 0.3496


Epoch 10, Loss: 0.3395
Epoch 10, Loss: 0.3395Epoch 10, Loss: 0.3395

Epoch 11, Loss: 0.3308Epoch 11, Loss: 0.3308Epoch 11, Loss: 0.3308


Epoch 12, Loss: 0.3238Epoch 12, Loss: 0.3238Epoch 12, Loss: 0.3238


Epoch 13, Loss: 0.3168Epoch 13, Loss: 0.3168Epoch 13, Loss: 0.3168


Epoch 14, Loss: 0.3116Epoch 14, Loss: 0.3116Epoch 14, Loss: 0.3116


Epoch 15, Loss: 0.3064Epoch 15, Loss: 0.3064Epoch 15, Loss: 0

[None, None, None]

In [18]:
# Install necessary libraries
!pip install crypten --no-deps
!pip install pandas scikit-learn torch torchvision

# Import libraries
import crypten
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Initialize CrypTen
crypten.init()

# Define the MPC workflow using the @crypten.mpc.run_multiprocess decorator
@crypten.mpc.run_multiprocess(world_size=3)
def train_mpc_model():
    # Load the Breast Cancer Wisconsin Dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
    data = pd.read_csv(url, header=None, names=columns)

    # Encode target variable: Malignant (M) = 1, Benign (B) = 0
    data['Diagnosis'] = data['Diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

    # Split features and labels
    X = data.iloc[:, 2:].values  # Features (30 columns)
    y = data['Diagnosis'].values  # Labels (binary classification)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Vertical splitting: divide features among 3 parties
    # Split X_train and X_test into 3 parts (columns split)
    X_party1 = X_train[:, :10]  # First 10 features
    X_party2 = X_train[:, 10:20]  # Next 10 features
    X_party3 = X_train[:, 20:]  # Last 10 features

    # Split test data similarly
    X_test_party1 = X_test[:, :10]
    X_test_party2 = X_test[:, 10:20]
    X_test_party3 = X_test[:, 20:]

    # Convert to PyTorch tensors
    X_party1_tensor = torch.tensor(X_party1, dtype=torch.float32)
    X_party2_tensor = torch.tensor(X_party2, dtype=torch.float32)
    X_party3_tensor = torch.tensor(X_party3, dtype=torch.float32)

    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test_party1_tensor = torch.tensor(X_test_party1, dtype=torch.float32)
    X_test_party2_tensor = torch.tensor(X_test_party2, dtype=torch.float32)
    X_test_party3_tensor = torch.tensor(X_test_party3, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Encrypt data using CrypTen for 3 parties
    encrypted_X_party1 = crypten.cryptensor(X_party1_tensor, src=0)
    encrypted_X_party2 = crypten.cryptensor(X_party2_tensor, src=1)
    encrypted_X_party3 = crypten.cryptensor(X_party3_tensor, src=2)
    encrypted_y_train = crypten.cryptensor(y_train_tensor, src=0)  # Labels shared only once

    # Combine encrypted data (horizontally stack features back together)
    encrypted_X_train = crypten.cat([encrypted_X_party1, encrypted_X_party2, encrypted_X_party3], dim=1)

    # Define logistic regression model using PyTorch
    class LogisticRegression(torch.nn.Module):
        def __init__(self, input_dim):
            super(LogisticRegression, self).__init__()
            self.linear = torch.nn.Linear(input_dim, 1)
            self.sigmoid = torch.nn.Sigmoid()

        def forward(self, x):
            return self.sigmoid(self.linear(x))

    # Create and encrypt the model
    model = LogisticRegression(input_dim=30)
    encrypted_model = crypten.nn.from_pytorch(model, torch.empty(1, 30))
    encrypted_model.encrypt()

    # Define loss function
    loss_fn = crypten.nn.BCELoss()

    # Set precision and learning parameters
    crypten.encoder.fixed_point_precision = 32
    learning_rate = 0.1
    num_epochs = 50

    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        predictions = encrypted_model(encrypted_X_train)
        loss = loss_fn(predictions, encrypted_y_train)

        # Backward pass
        encrypted_model.zero_grad()
        loss.backward()

        # Update parameters with learning rate decay
        lr = learning_rate / (1 + epoch * 0.1)  # Decay learning rate
        encrypted_model.update_parameters(lr)

        # Print decrypted loss for monitoring
        print(f"Epoch {epoch + 1}, Loss: {loss.get_plain_text().item():.4f}")

    # Combine encrypted test data
    encrypted_X_test = crypten.cat([crypten.cryptensor(X_test_party1_tensor, src=0),
                                     crypten.cryptensor(X_test_party2_tensor, src=1),
                                     crypten.cryptensor(X_test_party3_tensor, src=2)], dim=1)

    # Evaluate the model
    encrypted_model.eval()
    with torch.no_grad():
        predictions = encrypted_model(encrypted_X_test)
        decrypted_predictions = predictions.get_plain_text()
        binary_predictions = (decrypted_predictions >= 0.5).float()
        accuracy = (binary_predictions == y_test_tensor).float().mean().item()

    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Run the MPC model training
train_mpc_model()


Epoch 1, Loss: 0.7252Epoch 1, Loss: 0.7252Epoch 1, Loss: 0.7252


Epoch 2, Loss: 0.5720
Epoch 2, Loss: 0.5720
Epoch 2, Loss: 0.5720
Epoch 3, Loss: 0.4953Epoch 3, Loss: 0.4953Epoch 3, Loss: 0.4953


Epoch 4, Loss: 0.4490Epoch 4, Loss: 0.4490Epoch 4, Loss: 0.4490


Epoch 5, Loss: 0.4173Epoch 5, Loss: 0.4173Epoch 5, Loss: 0.4173


Epoch 6, Loss: 0.3936Epoch 6, Loss: 0.3936Epoch 6, Loss: 0.3936


Epoch 7, Loss: 0.3757Epoch 7, Loss: 0.3757
Epoch 7, Loss: 0.3757

Epoch 8, Loss: 0.3612Epoch 8, Loss: 0.3612
Epoch 8, Loss: 0.3612

Epoch 9, Loss: 0.3493Epoch 9, Loss: 0.3493Epoch 9, Loss: 0.3493


Epoch 10, Loss: 0.3395Epoch 10, Loss: 0.3395Epoch 10, Loss: 0.3395


Epoch 11, Loss: 0.3313Epoch 11, Loss: 0.3313Epoch 11, Loss: 0.3313


Epoch 12, Loss: 0.3237Epoch 12, Loss: 0.3237Epoch 12, Loss: 0.3237


Epoch 13, Loss: 0.3169Epoch 13, Loss: 0.3169Epoch 13, Loss: 0.3169


Epoch 14, Loss: 0.3114Epoch 14, Loss: 0.3114Epoch 14, Loss: 0.3114


Epoch 15, Loss: 0.3060Epoch 15, Loss: 0.3060Epoch 15, Loss: 0

[None, None, None]

In [19]:
# Install necessary libraries
!pip install crypten --no-deps
!pip install pandas scikit-learn torch torchvision

# Import libraries
import crypten
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Initialize CrypTen
crypten.init()

# Define the MPC workflow using the @crypten.mpc.run_multiprocess decorator
@crypten.mpc.run_multiprocess(world_size=3)
def train_mpc_model():
    # Load the Breast Cancer Wisconsin Dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
    data = pd.read_csv(url, header=None, names=columns)

    # Encode target variable: Malignant (M) = 1, Benign (B) = 0
    data['Diagnosis'] = data['Diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

    # Split features and labels
    X = data.iloc[:, 2:].values  # Features (30 columns)
    y = data['Diagnosis'].values  # Labels (binary classification)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Vertical splitting: divide features among 3 parties
    # Split X_train and X_test into 3 parts (columns split)
    X_party1 = X_train[:, :10]  # First 10 features
    X_party2 = X_train[:, 10:20]  # Next 10 features
    X_party3 = X_train[:, 20:]  # Last 10 features

    # Split test data similarly
    X_test_party1 = X_test[:, :10]
    X_test_party2 = X_test[:, 10:20]
    X_test_party3 = X_test[:, 20:]

    # Convert to PyTorch tensors
    X_party1_tensor = torch.tensor(X_party1, dtype=torch.float32)
    X_party2_tensor = torch.tensor(X_party2, dtype=torch.float32)
    X_party3_tensor = torch.tensor(X_party3, dtype=torch.float32)

    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test_party1_tensor = torch.tensor(X_test_party1, dtype=torch.float32)
    X_test_party2_tensor = torch.tensor(X_test_party2, dtype=torch.float32)
    X_test_party3_tensor = torch.tensor(X_test_party3, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Encrypt data using CrypTen for 3 parties
    encrypted_X_party1 = crypten.cryptensor(X_party1_tensor, src=0)
    encrypted_X_party2 = crypten.cryptensor(X_party2_tensor, src=1)
    encrypted_X_party3 = crypten.cryptensor(X_party3_tensor, src=2)
    encrypted_y_train = crypten.cryptensor(y_train_tensor, src=0)  # Labels shared only once

    # Combine encrypted data (horizontally stack features back together)
    encrypted_X_train = crypten.cat([encrypted_X_party1, encrypted_X_party2, encrypted_X_party3], dim=1)

    # Define logistic regression model using PyTorch
    class LogisticRegression(torch.nn.Module):
        def __init__(self, input_dim):
            super(LogisticRegression, self).__init__()
            self.linear = torch.nn.Linear(input_dim, 1)
            self.sigmoid = torch.nn.Sigmoid()

        def forward(self, x):
            return self.sigmoid(self.linear(x))

    # Create and encrypt the model
    model = LogisticRegression(input_dim=30)
    encrypted_model = crypten.nn.from_pytorch(model, torch.empty(1, 30))
    encrypted_model.encrypt()

    # Define loss function
    loss_fn = crypten.nn.BCELoss()

    # Set precision and learning parameters
    crypten.encoder.fixed_point_precision = 64
    learning_rate = 0.1
    num_epochs = 50

    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        predictions = encrypted_model(encrypted_X_train)
        loss = loss_fn(predictions, encrypted_y_train)

        # Backward pass
        encrypted_model.zero_grad()
        loss.backward()

        # Update parameters with learning rate decay
        lr = learning_rate / (1 + epoch * 0.1)  # Decay learning rate
        encrypted_model.update_parameters(lr)

        # Print decrypted loss for monitoring
        print(f"Epoch {epoch + 1}, Loss: {loss.get_plain_text().item():.4f}")

    # Combine encrypted test data
    encrypted_X_test = crypten.cat([crypten.cryptensor(X_test_party1_tensor, src=0),
                                     crypten.cryptensor(X_test_party2_tensor, src=1),
                                     crypten.cryptensor(X_test_party3_tensor, src=2)], dim=1)

    # Evaluate the model
    encrypted_model.eval()
    with torch.no_grad():
        predictions = encrypted_model(encrypted_X_test)
        decrypted_predictions = predictions.get_plain_text()
        binary_predictions = (decrypted_predictions >= 0.5).float()
        accuracy = (binary_predictions == y_test_tensor).float().mean().item()

    print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Run the MPC model training
train_mpc_model()


Epoch 1, Loss: 0.7248Epoch 1, Loss: 0.7248
Epoch 1, Loss: 0.7248

Epoch 2, Loss: 0.5725Epoch 2, Loss: 0.5725Epoch 2, Loss: 0.5725


Epoch 3, Loss: 0.4955Epoch 3, Loss: 0.4955Epoch 3, Loss: 0.4955


Epoch 4, Loss: 0.4488Epoch 4, Loss: 0.4488Epoch 4, Loss: 0.4488


Epoch 5, Loss: 0.4172
Epoch 5, Loss: 0.4172Epoch 5, Loss: 0.4172

Epoch 6, Loss: 0.3937Epoch 6, Loss: 0.3937Epoch 6, Loss: 0.3937


Epoch 7, Loss: 0.3762
Epoch 7, Loss: 0.3762Epoch 7, Loss: 0.3762

Epoch 8, Loss: 0.3615Epoch 8, Loss: 0.3615Epoch 8, Loss: 0.3615


Epoch 9, Loss: 0.3500Epoch 9, Loss: 0.3500Epoch 9, Loss: 0.3500


Epoch 10, Loss: 0.3398Epoch 10, Loss: 0.3398Epoch 10, Loss: 0.3398


Epoch 11, Loss: 0.3313
Epoch 11, Loss: 0.3313Epoch 11, Loss: 0.3313

Epoch 12, Loss: 0.3240Epoch 12, Loss: 0.3240Epoch 12, Loss: 0.3240


Epoch 13, Loss: 0.3173Epoch 13, Loss: 0.3173Epoch 13, Loss: 0.3173


Epoch 14, Loss: 0.3113Epoch 14, Loss: 0.3113Epoch 14, Loss: 0.3113


Epoch 15, Loss: 0.3063Epoch 15, Loss: 0.3063Epoch 15, Loss: 0

[None, None, None]